In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### Функция для сбора текстов со страницы

In [2]:
def get_article(url): #функция для парсинга страницы
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    collection = []
    content = soup.find_all('span', class_='ff2 fc0 fs12 fi')
    if '––§––' in content[0]:
        for item in content:
            articles = item.text.strip().split('––§––')
            for article in articles:
                collection.append(article)
    else:
        for article in content:
            if article.text != '––§––':
                collection.append(article.text.strip())
    return collection

### смотрю на ссылки, которые вообще есть на странице

In [3]:
pages = []
links = []
r = requests.get('https://skynight.ru/')
soup = BeautifulSoup(r.text, 'html.parser')
content = soup.find_all('span', class_='ff2 fc3 fs11')
for el in content:
    for page in el.find_all('a', class_='ImLink'):
        l = page.attrs['href']
        if not l[:4] == 'http':
            links.append(page.attrs['href']) #сохранила в список все ссылки со страницы

# сбор данных

In [5]:
def get_number_of_pages(url): #выясняю, сколько страниц в топике
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    page = soup.find_all('span', class_="ff2 fc4 fs14")[0].text
    pattern = r"\[\d\]"
    number_of_pages = len(re.findall(pattern, page))
    return number_of_pages

In [6]:
corpora = pd.DataFrame(columns=['text', 'topic', 'source'])

In [7]:
for link in links:
    n = get_number_of_pages(f'https://skynight.ru/{link}')
    topic = link.split('_')[0]
    for i in range (1, n+1):
        if i == 1:
                articles = get_article(f'https://skynight.ru/{link}')
                src = f'https://skynight.ru/{link}'
        else:
                articles = get_article(f'https://skynight.ru/{link[:-5]}_{i}.html')
                src = f'https://skynight.ru/{link[:-5]}_{i}.html'
        for article in articles:
            corpora.loc[len(corpora)] = [article, topic, src]


In [8]:
corpora

,text,topic,source
0,"Если кто-либо достоинНеземной любви,То я знаю,...",stihi,https://skynight.ru/stihi_love.html
1,"Спасибо за то, что ты есть.За то, что твой гол...",stihi,https://skynight.ru/stihi_love.html
2,"Среди таинственно-высоких гор,Вокруг кристальн...",stihi,https://skynight.ru/stihi_love.html
3,"Когда меня ты нежно обнимаешь,Я тут же забываю...",stihi,https://skynight.ru/stihi_love.html
4,"Люблю, скучаю, обожаю,Схожу с ума и умираюОт с...",stihi,https://skynight.ru/stihi_love.html
...,...,...,...
2147,Едут в купе три девушки и разговаривают между ...,anekdots,https://skynight.ru/anekdots_dating.html
2148,"Знакомство в Интернете.Парень: Скажи, а ты так...",anekdots,https://skynight.ru/anekdots_dating.html
2149,За столиком в кафе мужчина говорит женщине:– П...,anekdots,https://skynight.ru/anekdots_dating.html
2150,"– Девушка, поймите, я ведь не каждой предлагаю...",anekdots,https://skynight.ru/anekdots_dating.html


Добавляем цитаты:

In [9]:
url = f'https://2queens.ru/Articles/Biblioteka-Citaty-i-vyskazyvaniya/Citaty-iz-knig-o-lyubvi.aspx?ID=819'
r = requests.get(url).text
soup = BeautifulSoup(r, 'html.parser')
block = soup.find_all('blockquote')
for smth in block:
  texxt = smth.text.split()
  topic = 'quote'
  article = ' '.join(texxt)
  src = 'https://2queens.ru/Articles/Biblioteka-Citaty-i-vyskazyvaniya/Citaty-iz-knig-o-lyubvi.aspx?ID=819'
  corpora.loc[len(corpora.index )] = [article, topic, src]

In [14]:
corpora

,text,topic,source
0,"Если кто-либо достоинНеземной любви,То я знаю,...",stihi,https://skynight.ru/stihi_love.html
1,"Спасибо за то, что ты есть.За то, что твой гол...",stihi,https://skynight.ru/stihi_love.html
2,"Среди таинственно-высоких гор,Вокруг кристальн...",stihi,https://skynight.ru/stihi_love.html
3,"Когда меня ты нежно обнимаешь,Я тут же забываю...",stihi,https://skynight.ru/stihi_love.html
4,"Люблю, скучаю, обожаю,Схожу с ума и умираюОт с...",stihi,https://skynight.ru/stihi_love.html
...,...,...,...
2167,"…Я вся – одна сплошная любовь к тебе. Даже, по...",quote,https://2queens.ru/Articles/Biblioteka-Citaty-...
2168,"Нас нежно любят только женщины, на которых мы ...",quote,https://2queens.ru/Articles/Biblioteka-Citaty-...
2169,Он посмотрел на меня с ненавистью. Это была мн...,quote,https://2queens.ru/Articles/Biblioteka-Citaty-...
2170,Секреты… Ничто так не разъедает любовь.,quote,https://2queens.ru/Articles/Biblioteka-Citaty-...


Добавляем книги:

In [12]:
d = {'after.txt' : 'Анна Тодд, "После"', 'toalltheboys2.txt' : 'Дженни Хан, "P.S. Я все еще люблю тебя"', 'twilight.txt' : 'Стефани Майер, "Сумерки"'}

In [15]:
import os

In [16]:
trash = ['', '* * *', 'notes', 'Примечания']
list_of_files = os.listdir("books")
for book in list_of_files:
    with open ('books/' + book, 'r', encoding='utf-8') as f:
        text = f.read()
        text_split = text.split('\n')
        for abzats in text_split:
            if abzats not in trash and abzats.isupper() == False and re.match(r'(Глава|Часть) [0-9]*', abzats) == None:
                topic = 'book'
                article = abzats
                src = d[book]
                corpora.loc[len(corpora.index )] = [article, topic, src]

In [17]:
corpora

,text,topic,source
0,"Если кто-либо достоинНеземной любви,То я знаю,...",stihi,https://skynight.ru/stihi_love.html
1,"Спасибо за то, что ты есть.За то, что твой гол...",stihi,https://skynight.ru/stihi_love.html
2,"Среди таинственно-высоких гор,Вокруг кристальн...",stihi,https://skynight.ru/stihi_love.html
3,"Когда меня ты нежно обнимаешь,Я тут же забываю...",stihi,https://skynight.ru/stihi_love.html
4,"Люблю, скучаю, обожаю,Схожу с ума и умираюОт с...",stihi,https://skynight.ru/stihi_love.html
...,...,...,...
19023,"– Послушай, я люблю тебя больше всех на свете....",book,"Стефани Майер, ""Сумерки"""
19024,"– Да, – улыбнулся Эдвард. – Достаточно на сего...",book,"Стефани Майер, ""Сумерки"""
19025,Холодные губы снова прильнули к моей шее.,book,"Стефани Майер, ""Сумерки"""
19026,1,book,"Стефани Майер, ""Сумерки"""


In [18]:
corpora.to_csv('raw_data.csv', index = False) #записываем в файл